In [1]:
import numpy as np 
import pandas as pd  
from pandas.io.json import json_normalize 
import pymongo 

In [43]:
conn = pymongo.MongoClient(host='localhost', port=27017)
db = conn.project 
collection = db.boss_zhipin
data = collection.find()
data = json_normalize([comment for comment in data])
data.head(1)

,_id,bonus,company,degree,experience,job,job_desc,location,release_time,requirements,salary,staff_num
0,5c828202cf6cc807bc8e3eea,股票期权\t不打卡\t扁平管理\t领导nice\t公司氛围好\t带薪年假\t处女座公司\t年度旅游,投着乐,本科,1-3年,爬虫和页面分析提取专家,爬虫和页面分析提取研发专家7k-14k /杭州 / 经验3-5年 / 学历不限 / 全职职位...,杭州,2019-02-20 01:44,:1. 本科以上 从事java开发2～5年以上；2. 熟悉linux开发环境\t熟练使用sh...,7000-12000,0-20


In [44]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5267 entries, 0 to 5266
Data columns (total 12 columns):
_id             5267 non-null object
bonus           5267 non-null object
company         5267 non-null object
degree          5267 non-null object
experience      5267 non-null object
job             5267 non-null object
job_desc        5267 non-null object
location        5267 non-null object
release_time    5267 non-null object
requirements    3331 non-null object
salary          5267 non-null object
staff_num       5267 non-null object
dtypes: object(12)
memory usage: 493.9+ KB


In [ ]:
# 数据清洗  
data = data.drop(columns='_id')
data['release_time'] = pd.to_datetime(data['release_time'])
data = data.drop(data[data['job'] == ''].index)
data = data.fillna('')

In [49]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4671 entries, 0 to 5266
Data columns (total 11 columns):
bonus           4671 non-null object
company         4671 non-null object
degree          4671 non-null object
experience      4671 non-null object
job             4671 non-null object
job_desc        4671 non-null object
location        4671 non-null object
release_time    4671 non-null datetime64[ns]
requirements    4671 non-null object
salary          4671 non-null object
staff_num       4671 non-null object
dtypes: datetime64[ns](1), object(10)
memory usage: 437.9+ KB


In [118]:
title1 = data[data['job'].str.contains('爬虫')]['job'] 
title1.loc[title1.index] = '爬虫'
# data['title'] = title1
list(title1.index)
title1.dropna()

title2 = data[data['job'].str.contains('数据分析')]['job']
title2.loc[title2.index] = '数据分析'
# data['title'] = title2

# title = data[data['job'].str.contains('数据挖掘')]['job']
# title.loc[title.index] = '数据挖掘'
# data['title'] = title

C:\Users\Voyager\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Voyager\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


2

In [141]:
# 职位名称规范化 
indexs = list(data.index)
jobs = list(data.job)
titles = []
for job in jobs:
    if '爬虫' in job:
        titles.append('爬虫')
    elif '数据分析' in job:
        titles.append('数据分析')
    elif '数据挖掘' in job:
        titles.append('数据挖掘')
    else:
        titles.append('其他')
# dict(zip(jobs, titles))
title = dict(zip(indexs, titles))
title = pd.Series(title)
data['title'] = title
data['title'].value_counts()

数据分析    1676
其他      1321
数据挖掘    1015
爬虫       659
Name: title, dtype: int64

In [157]:
data.head(3)

,bonus,company,degree,experience,job,job_desc,location,release_time,requirements,salary,staff_num,title
0,股票期权\t不打卡\t扁平管理\t领导nice\t公司氛围好\t带薪年假\t处女座公司\t年度旅游,投着乐,本科,1-3年,爬虫和页面分析提取专家,爬虫和页面分析提取研发专家7k-14k /杭州 / 经验3-5年 / 学历不限 / 全职职位...,杭州,2019-02-20 01:44:00,:1. 本科以上 从事java开发2～5年以上；2. 熟悉linux开发环境\t熟练使用sh...,7000-12000,0-20,爬虫
1,,专家宝,本科,3-5年,大数据爬虫工程师,岗位职责：1、专注于垂直领域数据爬取\t进行多平台信息的抓取和分析； 2、参与爬虫系统的架构...,杭州,2019-02-21 14:28:00,：1、熟悉TCP/IP、HTTP、HTTPS等网络协议\t熟悉web前端\t可读懂js/cs...,25000-50000,0,爬虫
2,大数据\t机器学习\t免费零食\t公司氛围好\t电子商务\t不打卡,浙江云鉴数据科技,本科,1-3年,Python爬虫工程师(中级),【职位描述】1、负责大数据平台/核心项目的数据采集；升级scarpy-redis队列2、参与...,杭州,2018-11-16 16:30:00,】1、3年以上工作经验\t其中1年及以上实际的爬虫项目经验\t本科以上学历\t211或985...,15000-20000,20-99,爬虫


一、数据详情
* 共有3350条职位数据关于（爬虫，数据分析，数据挖掘）
* 包含的字段：福利，公司名，学历要求，工作经验，职位名称，职位描述，地点，发布时间，能力要求，薪水，员工人数，职位简称

二、问题
* 1.三种职位占比
* 2.职位地区分布
* 3.职位要求（学历，工作经验）
* 4.薪水的地区分布及区间分布
* 5.职位数与发布时间的关系 
* 6.工作内容
* 7.能力要求

In [155]:
data['title'].count()

3350

In [159]:
data = data.drop(data[data['title']=='其他'].index)
percentage = data['title'].value_counts() / data['title'].count()
percentage

数据分析    0.500299
数据挖掘    0.302985
爬虫      0.196716
Name: title, dtype: float64

In [183]:
from pyecharts import Pie 

indexes = list(data['title'].value_counts().index)
values = list(data['title'].value_counts().values)
pie = Pie('职位百分比')
pie.add('', indexes, values, radius=[40, 75], is_label_show=True)

## 2.职位地区分布

In [185]:
data.head(1)

,bonus,company,degree,experience,job,job_desc,location,release_time,requirements,salary,staff_num,title
0,股票期权\t不打卡\t扁平管理\t领导nice\t公司氛围好\t带薪年假\t处女座公司\t年度旅游,投着乐,本科,1-3年,爬虫和页面分析提取专家,爬虫和页面分析提取研发专家7k-14k /杭州 / 经验3-5年 / 学历不限 / 全职职位...,杭州,2019-02-20 01:44:00,:1. 本科以上 从事java开发2～5年以上；2. 熟悉linux开发环境\t熟练使用sh...,7000-12000,0-20,爬虫


In [192]:
from pyecharts import Bar

job_count_by_location = data['location'].value_counts()
bar = Bar('职位地区分布')
bar.add('', list(job_count_by_location.index), list(job_count_by_location.values), is_stack=True, is_label_show=True, 
       bar_category_gap='40%', label_color=['#130f40'])
bar

In [239]:
# 不同地区职位占比 
beijing = data[data['location']=='北京']['title'].value_counts()
# beijing
attr = list(beijing.index)
v = list(beijing.values)
# shanghai
shanghai = data[data['location']=='上海']['title'].value_counts()
attr1 = list(shanghai.index)
v1 = list(shanghai.values)
# hangzhou
hangzhou = data[data['location']=='杭州']['title'].value_counts()
attr2 = list(hangzhou.index)
v2 = list(hangzhou.values)
# shenzhen
shenzhen = data[data['location']=='深圳']['title'].value_counts()
attr3 = list(shenzhen.index)
v3 = list(shenzhen.values)
# chengdu
chengdu = data[data['location']=='成都']['title'].value_counts()
attr4 = list(chengdu.index)
v4 = list(chengdu.values)
# wuhan
wuhan = data[data['location']=='武汉']['title'].value_counts()
attr5 = list(wuhan.index)
v5 = list(wuhan.values)
# xiamen
xiamen = data[data['location']=='厦门']['title'].value_counts()
attr6 = list(xiamen.index)
v6 = list(xiamen.values)
# suzhou
suzhou = data[data['location']=='苏州']['title'].value_counts()
attr7 = list(suzhou.index)
v7 = list(suzhou.values)
# changshao
changsha = data[data['location']=='长沙']['title'].value_counts()
attr8 = list(changsha.index)
v8 = list(changsha.values)
# xian
xian = data[data['location']=='西安']['title'].value_counts()
attr9 = list(xian.index)
v9 = list(xian.values)

bar = Bar('不同地区职位占比', title_pos='auto')
bar.add('北京', attr, v)
bar.add('上海', attr, v1)
bar.add('杭州', attr, v2)
bar.add('深圳', attr, v3)
bar.add('成都', attr, v4)
bar.add('武汉', attr, v5)
bar.add('厦门', attr, v6)
bar.add('苏州', attr, v7)
bar.add('长沙', attr, v8)
bar.add('西安', attr, v9)

bar

### 3. 职位要求

In [259]:
##### degree = data['degree'].value_counts()
pie = Pie('学历要求')
pie.add('', list(degree.index), list(degree.values), is_label_show=True)
pie

In [265]:
# 不同职位学历要求 
pachong_degree = data[data['title']=='爬虫']['degree'].value_counts()
fenxi_degree = data[data['title']=='数据分析']['degree'].value_counts()
wajue_degree = data[data['title']=='数据挖掘']['degree'].value_counts()
pie = Pie('不同职位对学历的要求', title_pos='left', width=900)
pie.add('爬虫', list(pachong_degree.index), list(pachong_degree.values), center=[15, 50],
       is_random=True, radius=[20, 50], is_label_show=True)
pie.add('数据分析', list(fenxi_degree.index), list(fenxi_degree.values), center=[45, 50],
       is_random=True, radius=[20, 50], is_label_show=True)
pie.add('数据挖掘', list(wajue_degree.index), list(wajue_degree.values), center=[75, 50],
       is_random=True, radius=[20, 50], is_label_show=True)
pie

In [262]:
# 经验要求
experience = data['experience'].value_counts()
# experience
pie = Pie('经验要求', title_pos='left')
pie.add('', list(experience.index), list(experience.values), is_label_show=True)
pie

In [295]:
pachong_ex = data[data['title']=='爬虫']['experience'].value_counts()
# pachong_ex
fenxi_ex = data[data['title']=='数据分析']['experience'].value_counts()
wajue_ex = data[data['title']=='数据挖掘']['experience'].value_counts()
pie = Pie('不同职位对工作经验要求占比', width=1000)
pie.add('爬虫', list(pachong_ex.index), list(pachong_ex.values), is_label_show=True, center=[20, 50],radius=[20,50])
pie.add('数据分析', list(fenxi_ex.index), list(fenxi_ex.values), is_label_show=True, center=[50, 50],radius=[20,50])
pie.add('数据挖掘', list(wajue_ex.index), list(wajue_ex.values), is_label_show=True, center=[80, 50],radius=[20,50])
pie

## 4.薪水的地区分布及区间分布

In [284]:
# 薪水的区间分布 
salary = data['salary'].value_counts().head(10)
pie = Pie('薪水分布')
pie.add('', list(salary.index), list(salary.values), is_label_show=True, radius=[20,50])
pie

In [296]:
# 不同职位给出的薪水 
pachong_sa = data[data['title']=='爬虫']['salary'].value_counts().head(5)
fenxi_sa = data[data['title']=='数据分析']['salary'].value_counts().head(5)
wajue_sa = data[data['title']=='数据挖掘']['salary'].value_counts().head(5)
pie = Pie('不同职位给出的薪水', width=1000)
pie.add('爬虫', list(pachong_sa.index), list(pachong_sa.values), is_label_show=True, center=[20,50], radius=[15,40])
pie.add('数据分析', list(fenxi_sa.index), list(fenxi_sa.values), is_label_show=True, center=[55,50], radius=[15,40])
pie.add('数据挖掘', list(wajue_sa.index), list(wajue_sa.values), is_label_show=True, center=[85,50], radius=[15,40])
pie

In [318]:
from pyecharts import Line
data.set_index(data['release_time'], inplace=True)
# job_release_time = data['title'].resample('D').count()
job_release_time = data[data.index>pd.to_datetime('2019-1-1')]['title'].resample('D').count()
line = Line('职位发布与时间关系')
line.add('', list(job_release_time.index), list(job_release_time.values))
line

##  6. 工作内容

In [355]:
from pyecharts import WordCloud 
from collections import Counter 
import jieba

swords = [x.strip() for x in open('stopwords.txt', encoding='utf-8')]
def plot_wc(data):
    text = ''.join(data)
    words = list(jieba.cut(text))
    processed_words = []
    for word in words:
        if len(word) and (word not in swords):
            processed_words.append(word)
    c = Counter(processed_words)
    wc_data = pd.DataFrame({'keys': list(c.keys()), 'counts':list(c.values())}).sort_values(by='counts', ascending=False).head(100)
    wc = WordCloud(width=1000, height=500)
    wc.add('', wc_data['keys'], wc_data['counts'], words_size_range=[20,100])
    return wc

In [363]:
# data[data['title']=='数据分析']['job_desc']
# 岗位描述
plot_wc(data[data['title']=='数据分析']['job_desc'])
# data[data['title']=='数据分析']

In [362]:
# 技能要求
plot_wc(data[data['title']=='数据分析']['requirements'])